# Databricks notebook source

## 1. Libraries

In [ ]:
# https://www.nature.com/articles/s41598-022-15245-z
# https://www.nannyml.com/blog/91-of-ml-perfomance-degrade-in-time

In [ ]:
!pip install u8darts[all]==0.29.0
!pip install mlflow==2.11.3
dbutils.library.restartPython()

In [ ]:
# Darts
from darts import TimeSeries
from darts.models.forecasting.lgbm import LightGBMModel
from darts import TimeSeries
from darts import metrics
from darts.dataprocessing.transformers import Scaler

# Analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import plotly.graph_objs as go

# Utilities
import pandas as pd
import numpy as np
import datetime
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from datetime import timedelta
import datetime
import holidays
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import acf, pacf
from dateutil.relativedelta import relativedelta
import logging
from tqdm import tqdm
import pickle

# Notebook configuration
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_theme(style="whitegrid", palette="pastel")
#import warnings
#warnings.filterwarnings('ignore')


## 2. Data Import

In [ ]:
# Importing actual values
str_select_sales = '''
                        SELECT *
                        FROM analytics.refined_sales_orders_agg
                        '''

df_series_sales = spark.sql(str_select_sales).toPandas()

# Setting SYSTEM_TIMESTAMP column as DataFrame index
df_series_sales.set_index('SYSTEM_TIMESTAMP', inplace=True)
df_series_sales.sort_index(inplace=True)

# Converts granularity to Daily
df_series_sales_resampled = df_series_sales.resample('D').sum()

df_series_sales_resampled.info()

In [ ]:
# Importing predicted values
str_select_sales_preds = '''
                                SELECT *
                                FROM analytics.refined_sales_orders_forecast
                                '''

df_predictions = spark.sql(str_select_sales_preds).toPandas()
df_predictions.set_index('dt', inplace=True)
df_predictions.sort_index(inplace=True)

df_predictions.info()

In [ ]:
# Including actual values in predicted data
df_predictions['NET_VALUE_real'] = df_series_sales_resampled['NET_VALUE']
df_predictions.dropna(inplace=True)
df_predictions.sort_index(inplace=True)

df_predictions.info()

In [ ]:
# Importing existing metrics
str_select_metrics = '''
                                SELECT *
                                FROM analytics.refined_sales_orders_forecast_model_metrics
                                '''

df_current_metrics = spark.sql(str_select_metrics).toPandas()

df_current_metrics.info()


## 3. Calculating Metrics

In [ ]:
def calculate_wape(y, yhat):
    """
    Calculates Weighted Absolute Percentage Error (WAPE).
    
    Args:
        y (list or numpy array): Actual values.
        yhat (list or numpy array): Predicted values.
    
    Returns:
        float: WAPE value.
        
    Description:
    This function calculates the Weighted Absolute Percentage Error (WAPE).
    
    WAPE takes into account both absolute error and percentage error between actual and predicted values. 
    Furthermore, it performs error weighting based on actual values. In seasonal sales time series scenarios, 
    seasonal peak periods are assigned a higher weight, so errors in these periods have a more significant impact on this metric.
    
    Weighting is performed by assigning higher weights to larger actual values, since errors in high demand periods 
    can have a more relevant impact on planning decisions and financial results.
    
    In the code section where weight is defined as 'weight = actual', it is considered that the weight of each observation 
    is equal to the actual value of that observation itself. This means that when calculating WAPE, absolute percentage errors 
    are multiplied by the actual values themselves before being summed to calculate total weighted error.
    """
    
    # Initializes variables to store total WAPE and total weights
    total_wape = 0
    total_weight = 0
    
    try:
        # Loop over actual (y) and predicted (yhat) values
        for i in range(len(y)):
            actual = y[i]  # Actual value
            predicted = yhat[i]  # Predicted value
            
            # Calculates absolute error between actual and predicted value
            absolute_error = abs(actual - predicted)
            
            # Calculates absolute percentage error
            absolute_percentage_error = absolute_error / actual
            
            # Calculates weight as the actual value (used for weighting)
            weight = actual
            
            # Updates total WAPE summing weighted percentage error
            total_wape += absolute_percentage_error * weight
            
            # Updates total weights
            total_weight += weight
        
        # Calculates final WAPE as weighted average of percentage errors
        wape = total_wape / total_weight * 100
    except:
        # If an exception occurs (e.g., division by zero), sets WAPE as infinite
        wape = np.inf
        
    return wape

In [ ]:
# Separating necessary columns to calculate metrics
df_predictions_calc = df_predictions[['forecast_NET_VALUE_quartile_0_5', 'NET_VALUE_real', 'dt_predict']].rename(columns={'forecast_NET_VALUE_quartile_0_5': 'NET_VALUE_predicted'})
display(df_predictions_calc.reset_index())

In [ ]:
# Generating series
metrics_lists = {
    'mape': [], 'smape': [], 'ope': [], 'r2': [], 'rmse': [], 'wape': []
    }

# For each prediction batch, perform metric calculation
for dt_predict in df_predictions_calc.dt_predict.unique():
    
    # Creates series for metric calculation
    actual = TimeSeries.from_dataframe(df_predictions_calc[df_predictions_calc.dt_predict == dt_predict][['NET_VALUE_real']])
    predicted = TimeSeries.from_dataframe(df_predictions_calc[df_predictions_calc.dt_predict == dt_predict][['NET_VALUE_predicted']])

    # Calculates metrics
    mape = metrics.mape(actual + 1, predicted + 1)
    smape = metrics.smape(actual + 1, predicted + 1)
    ope = metrics.ope(actual + 1, predicted + 1)
    r2 = metrics.r2_score(actual + 1, predicted + 1)
    rmse = metrics.rmse(actual + 1, predicted + 1)
    wape = calculate_wape(actual + 1, predicted + 1)

    metrics_lists['mape'].append(mape)
    metrics_lists['smape'].append(smape)
    metrics_lists['ope'].append(ope)
    metrics_lists['r2'].append(r2)
    metrics_lists['rmse'].append(rmse)
    metrics_lists['wape'].append(wape.values().squeeze().item())

# Generates metrics dataframe
df_metrics = pd.DataFrame(metrics_lists)
df_metrics['dt_predict'] = pd.to_datetime(df_predictions_calc.dt_predict.unique())

In [ ]:
# Inspecting df_metrics
df_metrics.info()

In [ ]:
# Inspecting main metrics
display(df_metrics[df_metrics.dt_predict.dt.day_of_week < 5])

In [ ]:
# Inspecting actual vs predicted
display(df_predictions.reset_index())

In [ ]:
# Inspecting projection with historical actuals
display(pd.concat([df_series_sales_resampled.reset_index().iloc[-500:],
           df_predictions.reset_index().rename(columns={'dt': 'SYSTEM_TIMESTAMP'})], 
          axis=0))

In [ ]:
# Checking current model generation date
model_stage = 'staging'
mlflow_model_name = 'LightGBM_forecast_sales_daily'
client = MlflowClient()
model_metadata = client.get_latest_versions(mlflow_model_name, stages=[model_stage])

# Getting creation timestamp of latest version
creation_timestamp = model_metadata[0].creation_timestamp

# Formatting date
formatted_date = (datetime.datetime.fromtimestamp(creation_timestamp / 1000.0) - timedelta(hours=3)).strftime('%Y-%m-%d %H:%M')
formatted_date

In [ ]:
# Data for calculation
control_metric = 'wape'
df = df_metrics.set_index('dt_predict')[[control_metric]].sort_index()

# Calculation of process statistics
total_mean = df[control_metric].mean()
std_dev = df[control_metric].std()

# Calculate Moving Range (MR)
df['MR'] = df[control_metric].diff().abs()

# Calculate Moving Range mean
mr_mean = df['MR'][1:].mean()  # Ignora o primeiro valor que é NaN

# Control Limits for I-MR
UCL_I = total_mean + 3 * std_dev
LCL_I = total_mean - 3 * std_dev
UCL_MR = mr_mean * 3.267  # 3.267 is a constant for moving range in I-MR chart

# Functions for feature detection
def detect_sequence(data, cut_date, n=5):
    '''
    Detects existence of n consecutive points above the mean line.
    '''
    sequence_points = []
    count = 0
    for i, value in enumerate(data):
        if df.iloc[i].name > pd.Timestamp(cut_date):
            if value > total_mean:
                count += 1
                if count >= n:
                    sequence_points.extend(list(range(i - count + 1, i + 1)))
            else:
                count = 0
    return sequence_points

def detect_trend(data, cut_date, window=7):
    '''
    Detects upward or downward trends within a threshold.
    '''
    trend_points = []
    for i in range(len(data) - window + 1):
        if df.iloc[i].name > pd.Timestamp(cut_date):
            segment = data[i:i+window]
            if np.polyfit(range(len(segment)), segment, 1)[0] > 0:
                trend_points.append(i)
    return trend_points

def detect_control_limits(data, cut_date, UCL, LCL):
    '''
    Detects points outside control limits.
    '''
    out_of_control_points = []
    for i, point in enumerate(data):
        if df.iloc[i].name > pd.Timestamp(cut_date):
            if point > UCL or point < LCL:
                out_of_control_points.append(i)
    return out_of_control_points

# Feature detection
sequence_points = detect_sequence(df[control_metric], formatted_date)
trend_points = detect_trend(df[control_metric], formatted_date)
control_limit_points_I = detect_control_limits(df[control_metric], formatted_date, UCL_I, LCL_I)
control_limit_points_MR = detect_control_limits(df['MR'][1:], formatted_date, UCL_MR, 0)  # Ignores first value which is NaN

# Plotting Control Charts with Plotly
fig = go.Figure()

# Chart I (Individuals)
fig.add_trace(go.Scatter(x=df.index, y=df[control_metric], mode='lines+markers', name=control_metric))

# Add CL, UCL and LCL lines as horizontal lines
fig.add_shape(type='line',
              x0=df.index[0], y0=total_mean, x1=df.index[-1], y1=total_mean,
              line=dict(color='green', width=1, dash='dash'))
fig.add_shape(type='line',
              x0=df.index[0], y0=UCL_I, x1=df.index[-1], y1=UCL_I,
              line=dict(color='red', width=1, dash='dash'))
fig.add_shape(type='line',
              x0=df.index[0], y0=LCL_I, x1=df.index[-1], y1=LCL_I,
              line=dict(color='red', width=1, dash='dash'))

# Mark detected points
fig.add_trace(go.Scatter(x=df.index[sequence_points], y=df[control_metric].iloc[sequence_points],
                         mode='markers', name='Sequence', marker=dict(color='purple', symbol='circle')))
fig.add_trace(go.Scatter(x=df.index[trend_points], y=df[control_metric].iloc[trend_points],
                         mode='markers', name='Trend', marker=dict(color='orange', symbol='circle')))
fig.add_trace(go.Scatter(x=df.index[control_limit_points_I], y=df[control_metric].iloc[control_limit_points_I],
                         mode='markers', name='Out of Limits', marker=dict(color='black', symbol='circle')))

# Layout for Chart I
fig.update_layout(title='Control Chart I (Individuals)',
                  xaxis_title='Date',
                  yaxis_title=control_metric)

fig.show()


# Chart MR (Moving Range)
fig_MR = go.Figure()
fig_MR.add_trace(go.Scatter(x=df.index[1:], y=df['MR'].iloc[1:], mode='lines+markers', name='MR'))

# Add UCL_MR, CL_MR lines as horizontal lines
fig_MR.add_trace(go.Scatter(x=df.index[::], y=[UCL_MR]*len(df), mode='lines', line=dict(color='red', width=1, dash='dash'), name='UCL (MR)'))

# Add CL_MR line as horizontal line
fig_MR.add_trace(go.Scatter(x=df.index[::], y=[mr_mean]*len(df), mode='lines', line=dict(color='green', width=1, dash='dash'), name='CL (MR)'))

# Mark points outside limits in MR chart
fig_MR.add_trace(go.Scatter(x=df.index[1:][control_limit_points_MR], y=df['MR'].iloc[1:].iloc[control_limit_points_MR],
                            mode='markers', name='Out of Limits (MR)', marker=dict(color='black', symbol='circle')))

# Layout for Chart MR
fig_MR.update_layout(title='Control Chart MR (Moving Range)',
                     xaxis_title='Date',
                     yaxis_title='Moving Range')

# Show MR chart
fig_MR.show()

# Report of detected features
print("\nSequence Points:")
for point in df.iloc[sequence_points].index.values:
    print("-", str(pd.to_datetime(point)))

print("\nTrend Points:")
for point in df.iloc[trend_points].index.values:
    print("-", str(pd.to_datetime(point)))

print("\nPoints Out of Control Limits I:")
for point in df.iloc[control_limit_points_I].index.values:
    print("-", str(pd.to_datetime(point)))

print("\nPoints Out of Control Limits MR:")
for point in df.iloc[control_limit_points_MR].index.values:
    print("-", str(pd.to_datetime(point)))

In [ ]:
# Checking the need for model retraining
if len(sequence_points) > 0:
    print('Consecutive points above mean!')
    print('Retrain the model!')
    result = 'retrain'
else:
    print('No abnormality.')
    print('Model retraining unnecessary!')
    result = 'keep'

In [ ]:
# Adding calculated columns to dataframe
df['UCL_I'] = UCL_I
df['LCL_I'] = LCL_I
df['UCL_MR'] = UCL_MR
df['CL_MR'] = mr_mean
df['Flag_Out_Limit_I'] = 0
df['Flag_Out_Limit_MR'] = 0
df['Flag_Sequence'] = 0
df['Flag_Trend'] = 0

# Mark points out of control limits
df.loc[df.index[control_limit_points_I], 'Flag_Out_Limit_I'] = 1
df.loc[df.index[1:][control_limit_points_MR], 'Flag_Out_Limit_MR'] = 1

# Mark sequence points
df.loc[df.index[sequence_points], 'Flag_Sequence'] = 1

# Mark trend points
df.loc[df.index[trend_points], 'Flag_Trend'] = 1

In [ ]:
# Including statistical control data in metrics
df_metrics = df_metrics.merge(df.reset_index(drop=False).rename(columns={'SYSTEM_TIMESTAMP': 'dt_predict'}).drop(labels=[control_metric], axis=1), on='dt_predict', how='left')

In [ ]:
# Checking data to be saved
df_metrics[df_metrics.dt_predict > df_current_metrics.dt_predict.max()]

In [ ]:
# Saving data to table. Only new data
if len(df_metrics[df_metrics.dt_predict > df_current_metrics.dt_predict.max()]) > 0:
    df_spark = spark.createDataFrame(df_metrics[df_metrics.dt_predict > df_current_metrics.dt_predict.max()]) 
    mode = 'append' # overwrite or append
    overwriteSchema = 'False' # True or False
    table_name = 'analytics.refined_sales_orders_forecast_model_metrics'
    #path = '/dbfs/mnt/datalake/datascience/raw/forecast_sales/model_metrics/'

    df_spark.write.option("overwriteSchema", overwriteSchema).saveAsTable(table_name, 
                                                                        format='delta', 
                                                                        mode=mode,
                                                                        #path=f'{path}model_metrics')

In [ ]:
# Returns whether retraining should happen. This output is used by the Synapse pipeline to trigger the retraining notebook if necessary.
dbutils.notebook.exit(result)